In [14]:
import helpers
import json
import wowsims

Combinations of gear to try:

1. 2p t8 permutations: Head/Shoulder, Head/Chest, Head/Gloves, Head/Legs, Chest/Gloves, Chest/Legs, Chest/Gloves, Gloves/Legs
2. 2p t9 permutations: ^^

In [19]:
t8_slot_ids = [(0, 46161), (2, 46157), (4, 46159), (6, 46158), (8, 46160)]
t9_slot_ids = [(0, 48201), (2, 48198), (4, 48199), (6, 48202), (8, 48200)]
non_set_slot_ids = [(0, 47688), (1, 47060), (2, 45245), (3, 47545), (4, 47004), (5, 45611), (6, 47945), (7, 47112), (8, 45536), (9, 47077), (10, 47075), (11, 45608), (12, 45931), (13, 47131), (14, 47491), (16, 47668)]

In [20]:
t8_4pc = list(helpers.permute(t8_slot_ids, 4))
t9_4pc = list(helpers.permute(t9_slot_ids, 4))
t9_1pc = list(helpers.permute(t9_slot_ids, 1))
t8_2pc = list(helpers.permute(t8_slot_ids, 2))
t9_2pc = list(helpers.permute(t9_slot_ids, 2))
# Combinations of 2pc t8 and 2pc t9
t8_t9_4pc = []
for t8_pcs in t8_2pc:
    for t9_pcs in t9_2pc:
        if set([p[0] for p in t8_pcs]).intersection(set([p[0] for p in t9_pcs])):
            continue
        else:
            t8_t9_4pc.append(t8_pcs + t9_pcs)
# Item sets to swap with the base set
sets_to_consider = \
    list(zip(['t8_2/t9_2'] * len(t8_t9_4pc), t8_t9_4pc)) + \
    list(zip(['t8_2'] * len(t8_2pc), t8_2pc)) + \
    list(zip(['t9_2'] * len(t9_2pc), t9_2pc)) + \
    list(zip(['t8_4'] * len(t8_4pc), t8_4pc)) + \
    list(zip(['t9_4'] * len(t9_4pc), t9_4pc))

full_sets_to_consider = [('no_set', non_set_slot_ids)]
for (set_name, item_set) in sets_to_consider:
    altered_set = non_set_slot_ids.copy()
    replacements = dict(item_set)
    for index, item in enumerate(non_set_slot_ids):
        if item[0] in replacements:
            altered_set[index] = (item[0], replacements[item[0]])
    full_sets_to_consider.append((set_name, altered_set))

base_settings = json.load(open('data/p2_feral.json'))
del base_settings['raid']['parties'][0]['players'][0]['database']
del base_settings['raid']['parties'][0]['players'][0]['glyphs']['major1']

In [21]:
results = []
url = "https://wowsims.github.io/wotlk/feral_druid/#"
for (set_name, item_set) in full_sets_to_consider:
    settings = json.loads(json.dumps(base_settings))
    helpers.apply_item_set(settings, item_set)
    helpers.apply_gem_policy(settings)
    result = wowsims.runSim(settings)
    link_hash = wowsims.encodeSettings(settings)
    dps = result['raidMetrics']['dps']['avg']
    results.append((dps, url+link_hash, set_name))
results.sort(key=lambda x: x[0], reverse=True)

In [22]:
from IPython.display import display, HTML
list_items = [f'<tr><td><a href="{result[1]}">{result[2]}</a>:</td><td>{result[0]:0.0f}</td></tr>' for result in results]
display(HTML('<table>' + '\n'.join(list_items) + '</table>'))

"<a href=""https://wowsims.github.io/wotlk/feral_druid/#eJzsVDFoFE0Ynfft3v67yy+ZhIDjmTvWASEGArOzHmght2dleWBjI5ygqSyClVqdihC0VcGzEbEIWATESLY4FKLGKzxC0Cu0CXpKJMUdRGU7ZWcVazHpfMXHvPfmPQZmGH+vS4oOoU4NzKIJzAOrwDqwAYyx4mbBdTiKPfgjvJOS+FySzuNlaifke/z1gKS1mJD/P3+RklgvSattWLJF4uZE7o3wlZREWpLOYkJdYz/ok7hallbX2J1vJG5MGDvfPzck6XUNy4WXKYlBXtD7ngkfByRu/Y54/MOvk3h8vv9zafPNT9m8M8wiV7ZIvCpJZyGhhYR85tv80ReSu1xSdt0+YZ/HHHDRdiHoMpz62VMXzpy7hgPTlYORVjqMdKS1CpWqRKFWKlJRqCphOK1VVFGRCnUL7C3Ye4zz+30Sz59Q0HpDkytrpJbXaAP2EP/do6/kPyX2D3+FMfWniYfYn71zDlEIMIkjjAW1uHCsfJKxRq2JctHhpwUCyOuWZwLNmXg0j47He1q3M3Sq+2YMelX9jkQhOD61w1c5vBRvR02z82xbenYCh81sD+JaztNqA7nGVqtTlru0++ioi4BizOKu+ZuWgB8BAAD//88bjBQ="">t9_4:",12047
"<a href=""https://wowsims.github.io/wotlk/feral_druid/#eJzslMFrE0EUxud72ay7i5JpKDjGbFgHhFoozO4a0INk48ljwIsXIYL25KF4Uk9REYpeVTBeRDwUPBTESPcQFKqWCJZSNAe9BBul0kMCVdibsrP6B4jtzR/DY7735ns8mGGcgxYpOoYGNTGHFrAArAEDYBMostJW3jI5Sn04Bd5LSHxzpfl8mboxOTZ/PyKZ68Tk7OVvEhIDV+a6WsXbJO6Ws1qBryQkEleanZiy8pMhiZsVmVvV6m1C4k45UwU+PyZpr8bU0csp8sEGiZEr7SzR/5ke+jIica+sO3b0JBt/JrH5wvD31uBbX9P4YJxabmyTeOdKczGmxZgc5hj82XeS+yxSRsM4Y1zGPHDVsCDoOszGxXNXLly6hSMz1aNhoAI/DMIgUL5S1dAPlApV6Kuq788EKqyqUPlBG+wj2GdM8sdDEq9fkNf+QFMr66SW12kTxhh7HtEPcl4S+88/UVR/63iKw+k75xB5D1M4wZhXj/KnKmcZa9ZbqJRMfl7Ag7yds7WhNRtNZNbJ6ED7fkqvdmhW068Fn0jkvdPTu3yV42vRTrRp9V7tSJ/d4LiO3VFUz3RSayLLsbXadM5a2n9ywoJHEebwUP9NS8CvAAAA///6pYwi"">t9_4:",12030
"<a href=""https://wowsims.github.io/wotlk/feral_druid/#eJzsVD1oFEEYnffd3rq3KDc5Ao7n7bEuCDEQmJ31QAu5PSvLAxsb4QRNZRGs1OpUhKCtCp4WIhYBC0GMZItDIf5whRKCplCLw5wSsdiTqGyn7E60F3NdXvHxfe977zEww9i7LZJ0AE1qYQZtYA5YAvrAGlBi5W95y+Qof4Bd5L2ExBfHMx8vUjciu8DfxOTl5iOyt/MXCYm+4+W62RStk7he0bsSvx2TSByv0I1o5Rdp/f0BictVrS/y3k8S1yqeOR9l6yKfHVKm3yBKvL9KInb+cA8y0aeYxI2KZ77WRIGvpufR7dxgozX4189pvTVMLZfWSbxy/lqYbfBH38nbYZE0msYx4yxmgfOGBUEXYTZPnzh36swV7Juq7Q+UVH6gAqWkL2Ut8JWUgQx8WfP9KSWDmgykrzpg78A+YpzfG5B4/oTczluaeLlMcnGZ1mAMse0u/SD7KbEt/BdK8l8dD7E3fe0cIu9iAocYcxth/kj1OGOtRhvVsslPCrjwruYKmaE9HY5p63i4q3MzRa++ZzrDSl29J5F3j06O+CqHF8LNiGn3nm1KzihwMKvdOGzoOam3oDm2VJ/MWQs7D49ZcCnEDO5kP9QC8DsAAP//MROO0A=="">t9_2:",11972
"<a href=""https://wowsims.github.io/wotlk/feral_druid/#eJzslD9oFEEUxud7t7fuLspNjoDjmT3WBSEGArMzHmght2dleWBjI5ygqSyClVqdihC0VcHTQsQiYBEQI9lCFOK/E5QQ9Aptgp4SSbEHUdhO2V0vvZh0+TE8eN9832NghnH2WiTpEJrUwjTawCywBKwAq0CZVdaKlslR6cEp8W5C4ofrm08W6X1Ejs0/xOQX5iNydvJXCYkVd9hF6yRujuVdmd+NSSSub89H1PtNueNhn8TV6tD/NiFx46+/xGcGlJnz5ZT4m4RE7PrmhvAtJnFrbEOw+dfhSWw+2ye/MBeRY/C172m9M0gjV9ZJvHN9cy6idJM5Bn/8k/xdFkmjaZwwzmMGuGhYEHQZZvPsqQtnzl3DgcnaQa2kCrTSSslAypoOlJRa6kDWgmBSSV2TWgaqA/YJ7AtG+YM+iZfPyOt8pPHXyyQXl2kVxgA77tMvcp4T2+a/KMt/TTzC/vSdc4iih3EcYcxrhMVj1ZOMtRptVCsmPy3gwb9esLNAeyocyaOj4Z7O7ZRufd9URq+uPpMoescntvgqB5fCzRjT7r7YlDlbweGsPo3DRt4n9RZyjS3VJwrWwu6jIxY8CjGNe9nftAD8CQAA///9R4ub"">t9_4:",11967
"<a href=""https://wowsims.github.io/wotlk/feral_druid/#eJzsVEFoE0EUnfezWTeL0mkoOMZsWAeEWijMzhrQg2TjyWPAixchgvbkoXhST1ERil5VMF5EPBQ8CGJL9xAUqtYcDKVoQb0EG6UimECr5KbsrHoX25vv8PnvvXmfDzOMu9chRYdQozqm0QBmgWWgC6wDeVb4mnVsjsJbuCP85SaJz5605xepFZOb46/7JDNzMbk7+Yshia4nMy3D4g0SN4upN8KXhiSGnrTnYuoY+0GPxNWSzHSM3f5O4kbR2PNGmBmQzHUMS4Q8766R6Hu/tdUfyaGPfRK3itLuxPTQLLOWLJO2s71frcW/fErqnUESubJB4pX3J8Jciz/eJLnLIWXVrBPWecwAFy0Hgi7Drp09deHMuWs4MFk+GGqlg1CHWqtAqXIYaKVCFQaqHASTWoVlFapAN8HegX3AGL/fI/H8CfnNNzS+tEJqcYXWYQ2w4x59I/cpsf/4J+TV3yYeYX/y1DlE1sc4jjDmV6PssdJJxurVBkoFm58W8CGvZ3Im0JiKRtPoWLSneTtBu7JvymC1ot+TyPrHJ7b5KgeXoq0Y02g/25I524HDprb6UTXlw0odqcaWKxMZZ2H30VEHPkWYxl3zPS0APwMAAP//ZnaN0Q=="">t9_2:",11910
"<a href=""https://wowsims.github.io/wotlk/feral_druid/#eJzsVE1oE0EUnu9ls24WJdNQcIzZsC4ItVCYnTWgB8nGk8eAFy9CBO3JQ/GknqIiFL2qYPQg4qHgTWzpHkSh/lUwlKIF9RJslIrgBlphb8rOqngVm1s/hsf73nzf48EMz95tkaQDaFILU2gDM8AS0APWgBIrf8tbJkf5Hewif7lB4ovjmXML1I3ILvA3MXm52Yjs7fx5QqLneLmuZtE6ieuV7K7Eb8ckEscrdCNa+UGZ/n6fxOXqb/2rhMS1SsaKfHpAWjyrj13ivVUSsfN3rcg/xSRuVDyzG9GcHmY1HSZLZ/q/UoN//ZzGW4PUcmmdxGvnj4XZBn+4Qd4Oi6TRNI4ZZzENnDcsCLoIs3n6xLlTZ65g30Rtf6Ck8gMVKCV9KWuBr6QMZODLmu9PKBnUZCB91QF7D/YRo/xen8Szx+R23tLYi2WSC8u0BmOAbXfpO9lPiG3hv1CS/+p4gL3pV+cQ